In [ ]:
!python train_classifier_ssfl.py --data_name voc --model_name deeplab --control_name 500_fix@0.95-mix_3_0.5_iid_5-5_0.5_0_1

In [ ]:
git clone https://github.com/Karthik2924/SemiFL_extension.git

In [ ]:
gdown 1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65

In [ ]:
gdown 1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ

In [ ]:
pip install gdown torchmetrics anytree

In [10]:
from itertools import compress
import torch.nn as nn
import torch.nn.functional as F


In [5]:
import torch
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, num_samples=100, input_dim=5, output_dim=1):
        self.num_samples = num_samples
        self.data = torch.rand(100,3,128,128)
        self.target = torch.rand(100,1,128,128)


    def __len__(self):
        return 150
        #return self.num_samples

    def __getitem__(self, idx):
        idx = idx%100
        input_data, output_data = self.data[idx],self.target[idx]
        return input_data, output_data

# Create an instance of the ToyDataset
toy_dataset = ToyDataset(num_samples=100)


In [7]:
toy_dataset[110]

(tensor([[[0.4975, 0.7686, 0.4795,  ..., 0.1474, 0.7464, 0.2508],
          [0.5049, 0.9876, 0.5693,  ..., 0.5292, 0.8536, 0.0486],
          [0.9983, 0.2372, 0.6593,  ..., 0.3840, 0.9160, 0.1772],
          ...,
          [0.2423, 0.9949, 0.1096,  ..., 0.2962, 0.8126, 0.7632],
          [0.5282, 0.5487, 0.8986,  ..., 0.1457, 0.6552, 0.2331],
          [0.9383, 0.8419, 0.0575,  ..., 0.8684, 0.8314, 0.8974]],
 
         [[0.4601, 0.4387, 0.6227,  ..., 0.2938, 0.9770, 0.2080],
          [0.3307, 0.5525, 0.1654,  ..., 0.9443, 0.9633, 0.5328],
          [0.9122, 0.0173, 0.0084,  ..., 0.1135, 0.3655, 0.0996],
          ...,
          [0.7258, 0.5094, 0.5387,  ..., 0.6418, 0.5087, 0.6313],
          [0.3532, 0.8182, 0.7039,  ..., 0.9960, 0.9706, 0.4381],
          [0.5621, 0.8316, 0.0064,  ..., 0.6020, 0.5660, 0.3529]],
 
         [[0.0977, 0.4910, 0.7194,  ..., 0.8975, 0.3972, 0.3261],
          [0.3930, 0.2372, 0.8948,  ..., 0.8546, 0.6723, 0.8679],
          [0.2092, 0.3817, 0.5599,  ...,

In [ ]:
output_, input_ = {}, {}
output_['target'] = torch.cat(output, dim=0)
input_['target'] = torch.cat(target, dim=0)
#output_['target'] = F.softmax(output_['target'], dim=-1)
output_['target'] = F.softmax(output_['target'], dim=1)

In [17]:
op = torch.rand(12, 21, 256, 256)
ip = torch.randint(0,21,(12,256,256))
print(op.shape,ip.shape)

torch.Size([12, 21, 256, 256]) torch.Size([12, 256, 256])


In [18]:
op = F.softmax(op, dim=1)

In [19]:
op.shape

torch.Size([12, 21, 256, 256])

In [29]:
def make_hard_pseudo_label( soft_pseudo_label):
    #max_p, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1)
    max_p, hard_pseudo_label = torch.max(soft_pseudo_label, dim=1)
    #mask = max_p.ge(cfg['threshold'])
    mask = max_p.ge(0.08)
    return hard_pseudo_label, mask


In [30]:
nlab,mask = make_hard_pseudo_label(op)

In [33]:
torch.unique(mask,return_counts = True)

(tensor([False,  True]), tensor([752344,  34088]))

In [35]:
list(compress(nlab, mask))

RuntimeError: Boolean value of Tensor with more than one value is ambiguous